# Résumé de la tâche

On a plusieurs éléments sur lesquels travailler à ce stade : 
- un jeu de données contenant des questions et des réponses
- un ensemble de réponses à ces questions
- un corpus traitant de sujets médicaux

Nous disposons également de plusieurs pistes / contraintes :
- une indication du nombre de réponses possibles dans la question (systématique ?)
- la présence ou non d'une négation (on recherche ce qui est vrai ou ce qui est faux ?)
- le vocabulaire médical extrait de chaque question et chaque réponse

# Méthodologie

La première piste serait d'extraire les informations utiles des questions, que nous avons déjà mentionnées :
- nombre de réponses possibles
- négation ou affirmation ?
- vocabulaire médical

Une fois ces informations extraites des questions, on peut "unifier" le vocabulaire de la question avec celui de chaque réponse possible à la question, créant ainsi des ensembles :
- Q1 + R1 = E1
- Q1 + R2 = E2
- Q1 + R3 = E3
- Q1 + R4 = E4

Cela signifie lier un terme ou plusieurs termes médicaux (celui / ceux de la question) avec d'autres termes médicaux (ceux des réponses). Pour chaque ensemble, on regarde s'il existe dans le corpus une phrase (ou autre unité) qui lie les termes de cet ensemble grâce à une mesure de similarité en Ngrammes de char_wb. Une similarité au mot ferait peu de sens dans ce corpus en vue de la singularité des termes employés.

Avec les résultats obtenus, on crée un classement. On regarde ensuite si la question contient une négation et le nombre de réponses indiquées : 
- si on a 3 résultats à trouver, on prend les trois meilleurs ensembles selon les mesures de similarité
- si on a un résultat à trouver et une négation, on prend l'ensemble avec la pire mesure de similarité
- si on a 2 résultats et une négation, on prend les deux pires ensembles en terme de mesure de similarité
- ...

## Extraction du vocabulaire médical

Pour extraire le vocabulaire médical des questions et des réponses, plutôt que de trouver un corpus spécialisé assez exhaustif pour contenir tous les tecnolectes médicaux présents dans notre jeu de données, nous effectuons le raisonnement inverse : nous retrouvons les mots médicaux en supprimant tous les mots n'appartenant pas à ce domaine. Pour chaque question et pour chaque réponse, on a ainsi des listes de tecnolectes médicaux.

## Détection de la négation (pas terminée)

Il existe sûrement une solution avec Spacy (POS tagging -> étiquette "NEG" ?).
Attention : il peut exister des cas où la négation est présente mais n'indique pas une recherche de résultats "faux". 

## Nombre de bons résultats attendus

Voir avec Toufik.